In [14]:


import sys, os
from pyspark.sql.functions import col, explode, monotonically_increasing_id, lit, cast , concat, expr, date_format, sha2, concat_ws
from pyspark.sql.types import DateType, TimestampType   # sha2,concat_ws,trim, lit
from delta.tables import DeltaTable
from datetime import date, timedelta


In [13]:


# Insert the parent directory (one level up) onto Python’s module search path
sys.path.insert(0, os.path.abspath(".."))

from dev_spark_session import DevSparkSession 
from scd_type2_handler import SCDType2Handler
from python_helper import HelperMethons



### Initiate Spark

In [4]:

# stockdata = GetStockData() 
spark = DevSparkSession().spark


silver_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Silver/delta-table"  # Target location for Silver Delta table
dim_symbol_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Gold/delta-tables/dim-symbol"
fact_daily_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Gold/delta-tables/fact-daily-summary"



ERROR StatusLogger Reconfiguration failed: No configuration found for '5ffd2b27' at 'null' in 'null'
ERROR StatusLogger Reconfiguration failed: No configuration found for 'Default' at 'null' in 'null'


:: loading settings :: url = jar:file:/Users/PC/Desktop/VS%20Code%20Repositories/azure-stock-market/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/PC/.ivy2/cache
The jars for the packages stored in: /Users/PC/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-53f374a5-5f3c-4781-9390-c7beccbb2e20;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 75ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   

In [5]:

# silver_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Silver/delta-table"  # Target location for Silver Delta table

# df_silver = spark.read.format("delta").load(silver_path)

# df_silver.createOrReplaceTempView("stemp")

# spark.sql("select * from stemp").limit(2).show()


# silver_df = spark.read.format("delta").load(silver_path)

# spark.sql( f"delete from delta.`{silver_path}` where __Hashkey like '%c%' ") 

## Dim Symbol

In [6]:


parameters = {
        "businessColumns" : "Symbol,ExchangeName,Currency",
        "typeIColumns" : "", 
        "tableType" : "Dim"
        }
scd2Handler =  SCDType2Handler(parameters)




df_silver = spark.read.format("delta").load(silver_path)

df_dimSymbol =  df_silver.select("Symbol","ExchangeName","Currency", "Type", "ExchangeTimeZone").distinct()




scd2Handler.refresh_timestamp()

# df_dimSymbol = df_dimSymbol.withColumn("Type", concat(col("Type"), lit("_test") ) )


add_audit_columns =  scd2Handler.add_audit_columns


df_dimSymbol = df_dimSymbol.transform(add_audit_columns)



sid_offest = spark.read.format("delta").load(dim_symbol_path)


sid_offest = sid_offest.selectExpr("max(SymbolSID)").head()[0]

sid_offest = sid_offest + 1 if sid_offest else 0



df_dimSymbol = df_dimSymbol.withColumn("SymbolSID", monotonically_increasing_id() +  sid_offest)

spark.read.format("delta").load(dim_symbol_path).show()

df_dimSymbol.show(truncate=False)
deltaTable = DeltaTable.forPath(spark, dim_symbol_path)
scd2Handler.delta_merge_typeII(deltaTable, df_dimSymbol)




25/04/21 23:00:36 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+------+------------+--------+----+----------------+-------------+-------------+------------------------+----------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+----------------+
|SymbolSID|Symbol|ExchangeName|Currency|Type|ExchangeTimeZone|__CurrentFlag|__DeletedFlag|__EffectiveStartDateTime|__EffectiveEndDateTime|   __BusinessKeyHash|         __Hash1Type|         __Hash2Type| __CreatedBatchLogId|__UpdatedBatchLogId|    __CreateDateTime|__UpdateDateTime|
+---------+------+------------+--------+----+----------------+-------------+-------------+------------------------+----------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+----------------+
|        0|   VOO|        NYSE|     USD| ETF|America/New_York|         true|        false|    2025-04-21 22:51:...|   2099-12-31 00:00:00|362bd913e96f5481c..

25/04/21 23:00:41 WARN MergeIntoCommand: Merge source has SQLMetric(id: 1777, name: Some(number of source rows), value: 2) rows in initial scan but SQLMetric(id: 1778, name: Some(number of source rows (during repeated scan)), value: 0) rows in second scan


In [7]:
df_history = spark.sql(f" select *  from  delta.`{dim_symbol_path}`  ")
df_history = spark.sql(f" delete   from  delta.`{fact_daily_path}`  ")
# df_history = spark.sql(f" delete   from  delta.`{dim_symbol_path}`  ")

  
df_history.show(truncate=False)

+-----------------+
|num_affected_rows|
+-----------------+
|502              |
+-----------------+



## Fact Trading

In [12]:
df_silver = spark.read.format("delta").load(silver_path)
df_fact =  df_silver # .select("Volume","High","Low", "Close", "Open","Date")
df_fact =  df_fact.withColumn("DateID", date_format(col("Date"), "yyyyMMdd").cast("int")).drop("Date")


# Add Hash Key for Dim Symbol 

businessColumns  = ["Symbol", "ExchangeName", "Currency"] 

df_fact = df_fact.withColumn("DimSymbolBusinessHash" , sha2( concat_ws("|", *businessColumns), 256))



df_dim_symbol = spark.table(f"delta.`{dim_symbol_path}` ")

df_fact = df_fact.alias("f").join(df_dim_symbol.alias("d"), on = expr("f.DimSymbolBusinessHash = d.__BusinessKeyHash"), how = "left" ) \
                .where("d.__CurrentFlag = true") \
                .selectExpr("d.SymbolSID", 
                            "f.Volume", 
                            "f.High",
                            "f.Low",
                            "f.Open",
                            "f.Close",
                            "f.DateID"
                )



parameters = {
        "businessColumns" : "Symbol,ExchangeName,Currency",
        "typeIColumns" : "", 
        "tableType" : "Fact"
        }

scd2Handler =  SCDType2Handler(parameters)


scd2Handler.refresh_timestamp()
add_audit_columns =  scd2Handler.add_audit_columns
df_fact = df_fact.transform(add_audit_columns)


## ADD_SID

sid_offest = spark.read.format("delta").load(fact_daily_path)

sid_offest = sid_offest.selectExpr("max(TransactionSID)").head()[0]
sid_offest = sid_offest + 1 if sid_offest else 0



df_fact = df_fact.withColumn("TransactionSID", monotonically_increasing_id() +  sid_offest)



df_fact.show()

deltaTable = DeltaTable.forPath(spark, fact_daily_path)

scd2Handler.delta_merge_typeII(deltaTable, df_fact)




+---------+--------+----+---+----+-----+--------+-------------+--------------------+--------------------+--------------------+--------------+
|SymbolSID|  Volume|High|Low|Open|Close|  DateID|__DeletedFlag|       __FactKeyHash| __CreatedBatchLogId|    __CreateDateTime|TransactionSID|
+---------+--------+----+---+----+-----+--------+-------------+--------------------+--------------------+--------------------+--------------+
|        1|48899500| 390|380| 384|  389|20211221|        false|442a292c8e7230d8b...|0315c2ea-a41c-415...|2025-04-21 23:01:...|           502|
|        1|52545800| 398|393| 397|  398|20211210|        false|4ef81c23fa262901f...|0315c2ea-a41c-415...|2025-04-21 23:01:...|           503|
|        1|36664900| 337|333| 334|  337|20210409|        false|2c5399bcd95ab6984...|0315c2ea-a41c-415...|2025-04-21 23:01:...|           504|
|        1|33187200| 342|339| 342|  340|20210427|        false|10fe862d9b23ad3e7...|0315c2ea-a41c-415...|2025-04-21 23:01:...|           505|
|     

25/04/21 23:01:46 WARN MergeIntoCommand: Merge source has SQLMetric(id: 5301, name: Some(number of source rows), value: 502) rows in initial scan but SQLMetric(id: 5302, name: Some(number of source rows (during repeated scan)), value: 0) rows in second scan


In [11]:
df_history = spark.sql(f" select count(*)  from  delta.`{fact_daily_path}`  ")

# df_history = spark.sql(f" delete   from  delta.`{fact_daily_path}`  ")

  
df_history.show(truncate=False)

+--------+
|count(1)|
+--------+
|502     |
+--------+



In [ ]:


parameters = {
        "businessColumns" : "Symbol,ExchangeName,Currency",
        "typeIColumns" : "", 
        "tableType" : "Dim"
        }
scd2Handler =  SCDType2Handler(parameters)



silver_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Silver/delta-table"  # Target location for Silver Delta table

df_silver = spark.read.format("delta").load(silver_path)

df_dimSymbol =  df_silver.select("Symbol","ExchangeName","Currency", "Type", "ExchangeTimeZone").distinct()

dim_symbol_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Gold/delta-tables/dim-symbol"



scd2Handler.refresh_timestamp()

# df_dimSymbol = df_dimSymbol.withColumn("Type", concat(col("Type"), lit("_test") ) )

add_audit_columns =  scd2Handler.add_audit_columns
df_dimSymbol = df_dimSymbol.transform(add_audit_columns)



sid_offest = spark.read.format("delta").load(dim_symbol_path)


sid_offest = sid_offest.selectExpr("max(Symbol_SID)").head()[0]

sid_offest = sid_offest + 1 if sid_offest else 0



df_dimSymbol = df_dimSymbol.withColumn("Symbol_SID", monotonically_increasing_id() +  sid_offest)

spark.read.format("delta").load(dim_symbol_path).show()

df_dimSymbol.show(truncate=False)
deltaTable = DeltaTable.forPath(spark, dim_symbol_path)
scd2Handler.delta_merge_typeII(deltaTable, df_dimSymbol)




In [ ]:
from pyspark.sql.functions import sequence, to_date, explode, year, month, dayofmonth, expr, date_format, dayofweek, lit
from delta.tables import DeltaTable


In [ ]:



start_date = date.fromisoformat('2000-01-01')
end_date = date.fromisoformat('2010-01-01')


    # 1. Build a 1‑row DataFrame just so we can call sequence(…)
bounds = spark.range(1).withColumn("dates_list", sequence( lit(start_date), lit(end_date), expr("interval 1 day")))

bounds.show(truncate=False)

## Populate Dim Table if needed

In [15]:


hm = HelperMethons(spark=spark)
dimDate_folder =  "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Gold/delta-tables/dim-date"
df_dimdate = spark.read.format("delta").load(dimDate_folder)


max_date_dimDate =  df_dimdate.selectExpr(" cast(max(date) as date) as max_date").head()[0]
min_date_silver =  df_silver.selectExpr(" cast(min(date) as date) as min_date").head()[0]
max_date_silver =  df_silver.selectExpr(" cast(max(date) as date) as max_date").head()[0]



## populate DimDate table if we dont have records with dates 
if min_date_silver > max_date_dimDate:
    
    start_date = max_date_dimDate + timedelta(days=1)
    end_date =  date( max_date_silver.year , 12, 31)
    
    hm.update_DimDate_fromRange(start_date, end_date )
    print(f"Dim Table Updated with date range [ {start_date} : {end_date} ] ")
    
    
df_dimdate = spark.read.format("delta").load(dimDate_folder)




25/04/22 01:52:43 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 927054 ms exceeds timeout 120000 ms
25/04/22 01:52:43 WARN SparkContext: Killing executors is not supported by current scheduler.
25/04/22 02:10:10 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.B

In [ ]:

df.head(5)

### Get Data from API

### Bronze -> Silver